## Lipas data fetching

In this script Lipas data is fetched from geoserver. Then the data is saved to a file. After this, the saved data is used to fetch Mapple Insights reachabilities from the POIs.

In [53]:
# import necessary modules
import pandas as pd
import geopandas as gpd
import geojson
import shapely
import shapely.wkt
import json
import os
%run scripts # getGrid, getLipasData, getReachabilityDF, visualize
import mapclassify

Choose your municipality

In [54]:
municipality = input()

 Jyväskylä


In [55]:
# 1180 = frisbeegolf_rata
# 1340 = pallokenttä
# 2120 = kuntosali
# These will get data from lipas using typecode and typename
disc_golf_parks = getLipasData(typecode = '1180', 
                               typename = 'frisbeegolf_rata', 
                               municipality=municipality)
fitness_centers = getLipasData(typecode = '2120', 
                               typename = 'kuntosali', 
                               municipality=municipality)
ball_parks = getLipasData(typecode = '1340', 
                          typename = 'pallokentta', 
                          municipality=municipality)

Subset the ballparks to smaller size. Note that you do not have to execute this block for the program to work.

In [32]:
#ball_parks['kentan_pituus_m'] = pd.to_numeric(ball_parks['kentan_pituus_m'], downcast='integer')
#ball_parks = ball_parks.loc[ball_parks['kentan_pituus_m'] <= 50]

Save Lipas data to geojson files. This needs to be done in order to
fetch Mapple Insights data for the layers.

In [56]:
ball_parks.to_file("./input_folder/jkl_ball_parks.geojson", driver="GeoJSON")
disc_golf_parks.to_file("./input_folder/jkl_disc_golf.geojson", driver="GeoJSON")
fitness_centers.to_file("./input_folder/jkl_fitness_centers.geojson", driver="GeoJSON")

If you have not already, do the following commands on your terminal. Remember that the folder should be mapple-api-script, or the folder where main.py is located at, when executing the last 3 commands.

conda activate mapple-insights-api

export MAPPLE_API_KEY=YOUR_MAPPLE_API_KEY

python main.py -u https://staging.api.mapple.io -e ./input_folder/jkl_ball_parks.geojson -m 30 -d ./jkl_ball_parks_output_folder -i True

python main.py -u https://staging.api.mapple.io -e ./input_folder/jkl_disc_golf.geojson -m 30 -d ./jkl_disc_golf_output_folder -i True

python main.py -u https://staging.api.mapple.io -e ./input_folder/jkl_fitness_centers.geojson -m 30 -d ./jkl_fitness_centers_output_folder -i True

I have set -i to True so I can construct the result layers more easily.

Create 2 arrays that contain the names of the sports and traveling modes. And set the prefix accordingly.

In [65]:
sports = ['disc_golf', 'fitness_centers', 'ball_parks']
modes = ['walking', 'cycling', 'transit', 'driving']
prefix = input()

 jkl


This nested loop will create outputs as shapefiles in ./shapefiles-folder. The files are in folders named as sport_travelmode.

In [73]:
%run scripts # getGrid, getLipasData, getReachabilityDF, visualize
# Get the grid for joinging it with reachability-layer
grid = getGrid()
for sport in sports:
    for mode in modes:
        print(r'Processing {}_{}_output_folder'.format(prefix,sport), end='\r')
        output = getReachabilityDF(r'{}_{}_output_folder'.format(prefix,sport), r'{}'.format(mode),"")
        
        grid = grid.to_crs(outputs.crs)
        
        join = gpd.sjoin(output, grid, how="inner", op="intersects")
        join.to_file(r'./shapefiles/{}_{}_{}_grid'.format(prefix,sport,mode))

Now you should have reachability layers, as shapefiles, for the sport types you have chosen.